# Notebook to reproduce batch invoke bug, lighter version out of original.
Issue on github -> https://github.com/Azure/azure-sdk-for-python/issues/31166

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input, load_component
from azure.ai.ml.entities import (
    BatchEndpoint,
    PipelineComponentBatchDeployment,
)
from azure.core.exceptions import ResourceNotFoundError

WORKSPACES = {
    "dev_yml": {
        "subscription_id": "<SUBSCRIPTION HERE>",
        "resource_group_name": "<RESOURCE GROUP HERE>",
        "workspace_name": "<WORKSPACE NAME HERE>",
    },
}


ml_client = MLClient(DefaultAzureCredential(), **WORKSPACES["dev_yml"])
print(ml_client)

compute_cluster = "nc6-v3"

In [ ]:
try:
    ml_client.compute.get(compute_cluster)
except (ResourceNotFoundError, ValueError):
    raise BaseException("provided compute or credentials are not valid")

In [ ]:
endpoint_name = "collaborative-filteringhello"
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="collaborative-filteringhello",
)
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()
endpoint = ml_client.batch_endpoints.get(name=endpoint_name)
print(endpoint)

In [ ]:

pipeline_endpoint_component = load_component(
    source="./pipeline_endpoint_light.yml"
)

In [ ]:
deployment = PipelineComponentBatchDeployment(
    name="cf-train-save",
    description="collaborative-filtering-train-hello",
    endpoint_name=endpoint.name,
    component=pipeline_endpoint_component,
    settings={"continue_on_step_failure": False, "default_compute": compute_cluster},
)
ml_client.batch_deployments.begin_create_or_update(deployment).result()

# set default deployment for batch endpoint:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

In [ ]:
# invoke for sandbox:

interactions_date = "2023-06-18"
config_path = "aml/pipelines/collaborativefiltering/config/cf_owl_male.yaml"

job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name,
    inputs={
        "hello": Input(type="string", default="hello"),
        "interactions_date": Input(type="string", default=interactions_date),
        "config_path": Input(type="string", default=config_path),
        "extra_args": None,
    },
)
